In [1]:
import importlib
import Utils
import Chroma_func
import BM25_func
importlib.reload(Utils)  # This will print "Utils.py is loaded" again
importlib.reload(Chroma_func)
importlib.reload(BM25_func)


Utils.py is loaded
Utils.py is loaded
resource module not available on Windows
Utils.py is loaded
Utils.py is loaded


<module 'BM25_func' from 'c:\\gitProjects\\WebScraping\\BM25_func.py'>

In [ ]:
collection_name = "Pinkbike_articles"
corpus = BM25_func.get_corpus(collection_name)

querystr = "pump track"

bm25_dict = BM25_func.get_bm25_results(querystr, corpus, 10)


Split strings:   0%|          | 0/58 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/58 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/58 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/58 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 0.60): 10: BC Bike Race Introduces New eMTB Categories For 20
Rank 2 (score: 0.58): 35: Details Announced for the 2025 OneUp Squamish Endu
Rank 3 (score: 0.56): 25: Hot Laps Announces 2025 Event Details: PRESS RELEA
Rank 4 (score: 0.54): 12: Race Report: 2025 Southwest Regional Gravity Champ
Rank 5 (score: 0.53): 11: Event Report: The Inaugural Mt. Prevost DH Combine
Rank 6 (score: 0.52): 1: Podcast Round Up: Tahnée Seagrave on Speedsuits, K
Rank 7 (score: 0.52): 20: Race Report: New Zealand’s 2025 Enduro MTB Nationa
Rank 8 (score: 0.50): 37: Northwest Tune-Up Festival Announces 2025 Event De
Rank 9 (score: 0.50): 23: Race Report: Crankworx Rotorua Ends with a Bang Af
Rank 10 (score: 0.50): 56: Cannonball Mountain Bike Festival Kicks Off In Thr


In [3]:
bm25_ids = [item['id'] for item in bm25_dict]
bm25_ids

['6b04d7bdd514c0aaf823e89b121bfe039e52c17ffe5e4a3dec73c021a0078c8f',
 '3316c69fb1a945520b4e5c29b21f9d16b8bf8cc332d7d5e54d08a97e56060ce5',
 'ee07f8133068c83c300aa503f133b59bbfd5d48fd73f804496dc601908133738',
 '32c67a59d9d23dd6c4e3753a5c6ebf1345f63d1df6d9e0cf7c57289ab15fae64',
 '7bd833365ae5d35fa753388ea9cce7c3ff902fa3b857d5295ab81bd3fed97887',
 '30b33c692fc33bce7ad3931aab154b4c7d1c8ffd815c707c376bc994188ff462',
 '3b791beb904585e8ff462b989f1cf61b10fbe3d59cf9786c6e71c7f47202faa2',
 '01fb75263ffba4e1546078d8e42e678de50e7fe0a66ecf0dd552b20fe1fee642',
 'd2c6e602c400037c9e00f3c4d91b1b0d929bc3f538a8345271eea9a191c65c05',
 '2d0c2c36a1587b69a5ce9eeb58b2a6011f4178435622eee2d61d5f57d3a7ec39']

In [4]:
collection = Chroma_func.get_collection(collection_name)
chroma_vec_dict = Chroma_func.get_query_results(collection, querystr, 10)
vec_ids = chroma_vec_dict['ids'][0]
vec_ids

['90e22e54d23dd9cdf717ca98172740409d1c28f3e04da3db7e03f7b82e22b485',
 'ee07f8133068c83c300aa503f133b59bbfd5d48fd73f804496dc601908133738',
 'a0f8f0267b68ab1abb87e53ef450e038557a2ed4394dfecee43dd3c512bb0580',
 '8a5f5655ccd86e15bd8d63675203425dd5332601ad5ea3aa5ed05043eb00644d',
 '276143d0e530c1aec9f47729268ccb1a5480ad839f8d20595d35a5c47671cb68',
 'ab8ec7a6c92d3858b9998c1c70d8b1d4a217e4bc19baa86b57dc2f0339e5e409',
 '64a249b7c7176da01c42c6446558d6e3d6b4499be1e0b6d279768a91a5264f4b',
 '6b04d7bdd514c0aaf823e89b121bfe039e52c17ffe5e4a3dec73c021a0078c8f',
 '7bd833365ae5d35fa753388ea9cce7c3ff902fa3b857d5295ab81bd3fed97887',
 '8f6a9aa3dc1d9b482637dc3b0397a978cd89be2d12c17d8592a06840b4bfa9e2']

In [5]:
# https://gist.github.com/srcecde/eec6c5dda268f9a58473e1c14735c7bb

from collections import defaultdict
def reciprocal_rank_fusion(*list_of_list_ranks_system, K=60):
    """
    Fuse rank from multiple IR systems using Reciprocal Rank Fusion.
    
    Args:
    * list_of_list_ranks_system: Ranked results from different IR system.
    K (int): A constant used in the RRF formula (default is 60).
    
    Returns:
    Tuple of list of sorted documents by score and sorted documents
    """
    # Dictionary to store RRF mapping
    rrf_map = defaultdict(float)

    # Calculate RRF score for each result in each list
    for rank_list in list_of_list_ranks_system:
        for rank, item in enumerate(rank_list, 1):
            rrf_map[item] += 1 / (rank + K)

    # Sort items based on their RRF scores in descending order
    sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

    # Return tuple of list of sorted documents by score and sorted documents
    return sorted_items, [item for item, score in sorted_items]


# Example ranked lists from different sources
ir_system_a = ['Document1', 'Document3', 'Document5', 'Document7']
ir_system_b = ['Document2', 'Document1', 'Document4']
ir_system_c = ['Document5', 'Document3', 'Document2']

# Combine the lists using RRF
combined_list = reciprocal_rank_fusion(ir_system_a, ir_system_b, ir_system_c)
print(combined_list)

([('Document1', 0.03252247488101534), ('Document5', 0.032266458495966696), ('Document2', 0.032266458495966696), ('Document3', 0.03225806451612903), ('Document4', 0.015873015873015872), ('Document7', 0.015625)], ['Document1', 'Document5', 'Document2', 'Document3', 'Document4', 'Document7'])


In [6]:
# Combine the lists using RRF
combined_list = reciprocal_rank_fusion(bm25_ids, vec_ids)
print(combined_list[1])

['ee07f8133068c83c300aa503f133b59bbfd5d48fd73f804496dc601908133738', '6b04d7bdd514c0aaf823e89b121bfe039e52c17ffe5e4a3dec73c021a0078c8f', '7bd833365ae5d35fa753388ea9cce7c3ff902fa3b857d5295ab81bd3fed97887', '90e22e54d23dd9cdf717ca98172740409d1c28f3e04da3db7e03f7b82e22b485', '3316c69fb1a945520b4e5c29b21f9d16b8bf8cc332d7d5e54d08a97e56060ce5', 'a0f8f0267b68ab1abb87e53ef450e038557a2ed4394dfecee43dd3c512bb0580', '32c67a59d9d23dd6c4e3753a5c6ebf1345f63d1df6d9e0cf7c57289ab15fae64', '8a5f5655ccd86e15bd8d63675203425dd5332601ad5ea3aa5ed05043eb00644d', '276143d0e530c1aec9f47729268ccb1a5480ad839f8d20595d35a5c47671cb68', '30b33c692fc33bce7ad3931aab154b4c7d1c8ffd815c707c376bc994188ff462', 'ab8ec7a6c92d3858b9998c1c70d8b1d4a217e4bc19baa86b57dc2f0339e5e409', '3b791beb904585e8ff462b989f1cf61b10fbe3d59cf9786c6e71c7f47202faa2', '64a249b7c7176da01c42c6446558d6e3d6b4499be1e0b6d279768a91a5264f4b', '01fb75263ffba4e1546078d8e42e678de50e7fe0a66ecf0dd552b20fe1fee642', 'd2c6e602c400037c9e00f3c4d91b1b0d929bc3f538a834

In [7]:
result = collection.get(
    ids=combined_list[1]  # Pass in the list of document IDs
)
result

{'ids': ['30b33c692fc33bce7ad3931aab154b4c7d1c8ffd815c707c376bc994188ff462',
  '64a249b7c7176da01c42c6446558d6e3d6b4499be1e0b6d279768a91a5264f4b',
  '6b04d7bdd514c0aaf823e89b121bfe039e52c17ffe5e4a3dec73c021a0078c8f',
  '7bd833365ae5d35fa753388ea9cce7c3ff902fa3b857d5295ab81bd3fed97887',
  '32c67a59d9d23dd6c4e3753a5c6ebf1345f63d1df6d9e0cf7c57289ab15fae64',
  '276143d0e530c1aec9f47729268ccb1a5480ad839f8d20595d35a5c47671cb68',
  '90e22e54d23dd9cdf717ca98172740409d1c28f3e04da3db7e03f7b82e22b485',
  '3b791beb904585e8ff462b989f1cf61b10fbe3d59cf9786c6e71c7f47202faa2',
  'a0f8f0267b68ab1abb87e53ef450e038557a2ed4394dfecee43dd3c512bb0580',
  'd2c6e602c400037c9e00f3c4d91b1b0d929bc3f538a8345271eea9a191c65c05',
  'ee07f8133068c83c300aa503f133b59bbfd5d48fd73f804496dc601908133738',
  '3316c69fb1a945520b4e5c29b21f9d16b8bf8cc332d7d5e54d08a97e56060ce5',
  '01fb75263ffba4e1546078d8e42e678de50e7fe0a66ecf0dd552b20fe1fee642',
  '8f6a9aa3dc1d9b482637dc3b0397a978cd89be2d12c17d8592a06840b4bfa9e2',
  'ab8ec7a6c9

In [8]:
# Re-order documents based on the original order of combined_list[1]
ordered_documents = []
for doc_id in combined_list[1]:
    # Find the document in the result that matches the current doc_id
    for idx, doc in enumerate(result['documents']):
        if result['ids'][idx] == doc_id:
            ordered_documents.append(doc)
            break

# Display the ordered documents
for idx, doc in enumerate(ordered_documents):
    print(idx+1, doc)

1 Hot Laps Announces 2025 Event Details: PRESS RELEASE: Hot Laps MTB Dates: May 30 - June 1, 2025 Location: Port Gamble Heritage Park, WA Created By: Jill Kintner + Matt Orlando + Team Hot Laps is a 3-day celebration of mountain biking, where friendly competition, progression, and good times take center stage. Racing shouldn't be about pre-race porta-potty panic, lung-searing torture, or legs full of lactic acid. It’s not even about the sweet taste of victory or the sting of defeat. It’s about the people—your ride-or-die crew who cheer you on, push you harder and make you laugh when you’re sucking wind. At Hot Laps, we believe in "NO BAD TIMES!" Founded by Jill Kintner (Olympic medalist, World Champion) and Matt Orlando (trail builder, videographer), Hot Laps is a weekend created by riders, for riders with a focus on community and process—challenging yourself with short, repeatable laps, get multiple chances to race your best, and sessions with friends. Whether you're racing, demoing b